# 05 범용 고유 식별자

* SW개발 시엔 다양한 형태의 식별자가 필요하다. 큰 작업을 여러 작업으로 나눌 때 각 작업의 진행 상태를 추적할 식별자(작업 ID), 서비스 이용자를 구분하기 위한 식별자(유저 ID), 클라이언트가 연속적인 요청을 보낸 것인지 새로운 요청을 보낸 것인지 구분을 위한 식별자(세션 ID) 등
* 이런 식별자에 범용적으로 사용하는 것이 Universally Unique IDentifier : UUID이다.
* 범용 고유 식별자란 컴퓨터 시스템 내에 고유한 객체를 식별하기 위해 사용하는 값이다.

### 5.1 UUID 구조
* UUID는 하이픈 4개, 16진수 32개로 구성되어 있으며, 구조는 `8개-4개-4개-4개-12개`이다.
* 하이픈을 생략하는 경우도 많다.
* UUID 값을 저장할 때는 하이픈을 제외한 16진수만 저장한다. 16진수를 하나 표현하는 데 4비트(0.5바이트)이므로 총 32*4 = 128비트(16바이트)가 필요하다.
* UUID는 32개의 16진수 모두 무작위로 만든다. 100경 개의 UUID를 생성했을 때 1쌍의 UUID가 충돌할 확률은 50%이다. 즉, 같은 UUID가 나올 확률은 매우 낮다.
* 많은 식별자가 필요하지 않은 경우, 일부 값만 생성해서 사용하고 나머지는 고정 값을 사용한다. 예를 들어, UUID의 앞부분은 고정하고 뒷부분 12글자만 생성하여 맥 주소로 사용할 수 있다.

### 5.2 UUID 버전
* UUID는 버전별로 방법과 규칙이 다르다.
* 오늘날은 주로 버전 4 UUID나 이를 기반으로 직접 정의한 UUID를 사용한다.

### 버전1과 버전2
* 초창기 버전의 UUID로 네트워크에 있는 고유한 컴퓨터들을 식별할 때 사용했음. 버전1은 컴퓨터의 맥 주소와 타임스탬프 값을 기반으로 UUID를 생성했고, 버전2는 자릿수 일부를 지역별 정의 site-defined로 남겨뒀다.
* 맥 주소와 타임스탬프 값으로 서버를 식별하는 것은 한계가 있으므로, 버전 1, 2는 더 이상 사용하지 않는다. (ex 마이크로 서비스 아키텍처는 맥 주소가 달라도 같은 서버로 인식해야 한다. 도커와 같은 가상 서버는 같은 맥 주소를 가진 여러 서버를 동시에 생성할 수 있다.)
  
### 버전3과 버전5
* 네임스페이스와 문자열을 받아 해시 함수를 사용해 UUID로 변환한다. 따라서 입력값이 같으면 UUID도 같으며, 해시 충돌 방지를 위해 별도의 네임스페이스 UUID를 받는다.
* 버전 3의 UUID는 MD3를, 버전 5는 SHA-1를 기반으로 한다.
* MD5는 취약점 발견 이후 사용하지 않으나, 시스템 내부에서 고유 데이터 식별 용도엔 문제가 없다.
  
### 버전4
* 버전 표기를 위한 4비트를 제외하곤 나머지 122비트는 모두 무작위로 생성한다.
* 가장 많이 사용하는 UUID 버전이다.

In [1]:
#!/usr/bin/python3
import uuid

for i in range(1, 5):
    print('랜덤 UUID({0})={1}'.format(i, uuid.uuid4()))


랜덤 UUID(1)=f55ca695-e2c1-4bfb-b537-65e3b2bdcbd9
랜덤 UUID(2)=ae5b5791-a83e-436c-99f1-c7380f0e04b6
랜덤 UUID(3)=52562ba4-2187-4a44-b19f-36dcf8ab1bf9
랜덤 UUID(4)=f7498d58-3c18-447a-a755-e6bbbaddcd88


### UUID 사용하기
* UUID는 논리적인 작업의 고유 식별자로 쓰기 적합하다.
* 논리적인 작업 : 웹 페이지, RESTful API 요청, 파일 업로드, 상품 주문 처리 등
* 상품 주문, 결제 등의 작업은 길게는 몇 시간부터 며칠이 걸린다. 작업 시간이 길어질수록 프로세스나 사용하는 함수가 많아진다. 따라서 복잡한 작업 추적을 위해 고유한 ID가 필요하다.

In [2]:
# 웹 페이지 요청마다 UUID 생성

import uuid
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello():
    request_id = uuid.uuid4()
    print('API 요청 ID={0}'.format(request_id))
    return 'Hello World'

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2022 01:10:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 01:10:11] "GET /favicon.ico HTTP/1.1" 404 -


API 요청 ID=c1dfc9ae-1b45-4092-b3a4-f53bd1af483a


127.0.0.1 - - [14/Aug/2022 01:10:20] "GET / HTTP/1.1" 200 -


API 요청 ID=0d8f826d-bd6c-499b-8872-542d0aa5a634


* 새로운 요청이 올 때마다 ID가 바뀌는 것을 볼 수 있다. 이렇게 생성한 UUID는 요청이 끝나기 전이나 논리적인 작업이 끝나기 전까지 가지고 있거나 DB에 넣어 관리할 수 있다.
* 낮은 확률로 UUID 충돌이 발생할 수 있다. ETCD, Redis와 같은 서비스를 이용해 먼저 UUID를 생성한 쪽만 사용할 수 있게 하는 방법이 있다.

### 고정된 값을 포함하는 UUID

In [3]:
# 앞 8자리를 타임스탬프 값으로 바꿔 사용
import uuid
import time

now = str(int(time.time()))
print("현재 시간(Unix epoch time): {0}".format(now))

uuid_str = str(uuid.uuid4())
print("생성된 UUID={0}".format(uuid_str))

new_uuid_str = now[0:8] + "-" + now[8:10] + "00-" + uuid_str[14:]
print("새로 만든 UUID={0}".format(new_uuid_str))

현재 시간(Unix epoch time): 1660407095
생성된 UUID=c6d78bb7-f70e-4d4d-ab80-37fdc5a6fcc7
새로 만든 UUID=16604070-9500-4d4d-ab80-37fdc5a6fcc7


### UUID의 단점
* UUID는 16바이트나 사용한다. 이는 비효율적인 때가 있다. 예로, 1씩 증가하는 게시판 글번호 값보다 UUID는 비효율적이다. DB의 Key로 UUID를 사용한다면, 4바이트를 사용하는 식별자보다 검색, 추가, 삭제 속도가 느릴 수 있다.
* UUID는 무작위로 생성된 글자여서 UUID만 보고 구체적인 내용을 알기가 어렵다.